In [ ]:
# importing dependencies
! pip install langchain milvus pymilvus python-dotenv sentence_transformers

In [ ]:
# initializing
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
import os

from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("NEBULA_KEY")

In [ ]:
# embeddings generation
from milvus import default_server
default_server.start()

from langchain_community.embeddings import HuggingFaceEmbeddings
# is this model by default: sentence-transformers/all-mpnet-base-v2
embeddings = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
!pip uninstall grpcio -y

Found existing installation: grpcio 1.60.0
Uninstalling grpcio-1.60.0:
  Successfully uninstalled grpcio-1.60.0


In [ ]:
!pip install grpcio==1.60.0

  Using cached grpcio-1.60.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.4 MB)


In [ ]:
!pip install --upgrade pymilvus

In [ ]:
# hosting and collection normalization
from pymilvus import utility, connections

connections.connect(host="127.0.0.1", port=default_server.listen_port)
utility.drop_collection('LangChainCollection')

In [ ]:
# api model store
from langchain.vectorstores import Milvus
vectordb = Milvus.from_documents(
   {},
   embeddings,
   connection_args={"host": "127.0.0.1", "port": default_server.listen_port},
   consistency_level="Strong")

In [ ]:
# information retrieve
retriever = Milvus.as_retriever(vectordb, search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)
about_me = [
   {"input": "My favorite snack is chocolate",
    "output": "Nice"},
   {"input": "My favorite sport is swimming",
    "output": "Cool"},
   {"input": "My favorite beer is Guinness",
    "output": "Great"},
   {"input": "My favorite dessert is cheesecake",
    "output": "Good to know"},
   {"input": "My favorite musician is Taylor Swift",
    "output": "I also love Taylor Swift"}
]
for example in about_me:
   memory.save_context({"input": example["input"]}, {"output": example["output"]})

In [ ]:
print(memory.load_memory_variables({"prompt": "who is my favorite musician?"})["history"])

input: My favorite musician is Taylor Swift
output: I also love Taylor Swift
